In [8]:
import pandas as pd
import numpy as np 
import re
import string
from nltk.corpus import stopwords

In [9]:
df = pd.read_csv("C:/Users/Admin/NLP_Project/Text_Preprocessing and Classification/Movies_review_dataset/IMDB Dataset.csv")
df = pd.DataFrame(df.iloc[:2000])
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
1995,"Feeling Minnesota, directed by Steven Baigelma...",negative
1996,THE CELL (2000) Rating: 8/10<br /><br />The Ce...,positive
1997,"This movie, despite its list of B, C, and D li...",negative
1998,I loved this movie! It was all I could do not ...,positive


In [10]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [11]:
def remove_htmltag(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

df['review'] = df['review'].apply(remove_htmltag).str.lower()

In [12]:
print(df)

                                                 review sentiment
0     one of the other reviewers has mentioned that ...  positive
1     a wonderful little production. the filming tec...  positive
2     i thought this was a wonderful way to spend ti...  positive
3     basically there's a family where a little boy ...  negative
4     petter mattei's "love in the time of money" is...  positive
...                                                 ...       ...
1995  feeling minnesota, directed by steven baigelma...  negative
1996  the cell (2000) rating: 8/10the cell, like ant...  positive
1997  this movie, despite its list of b, c, and d li...  negative
1998  i loved this movie! it was all i could do not ...  positive
1999  this was the worst movie i have ever seen bill...  negative

[2000 rows x 2 columns]


In [13]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

df['review'] = df['review'].apply(remove_url)

In [14]:
def remove_punctuation(text):
    return text.translate(str.maketrans('','', string.punctuation))

df['review'] = df['review'].apply(remove_punctuation)
print(df)

                                                 review sentiment
0     one of the other reviewers has mentioned that ...  positive
1     a wonderful little production the filming tech...  positive
2     i thought this was a wonderful way to spend ti...  positive
3     basically theres a family where a little boy j...  negative
4     petter matteis love in the time of money is a ...  positive
...                                                 ...       ...
1995  feeling minnesota directed by steven baigelman...  negative
1996  the cell 2000 rating 810the cell like antz mus...  positive
1997  this movie despite its list of b c and d list ...  negative
1998  i loved this movie it was all i could do not t...  positive
1999  this was the worst movie i have ever seen bill...  negative

[2000 rows x 2 columns]


In [15]:
def remove_stopwords(text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append(" ")
        else:
            new_text.append(word)
    return ' '.join(new_text)

df['review'] = df['review'].apply(remove_stopwords)
print(df)

                                                 review sentiment
0     one       reviewers   mentioned     watching  ...  positive
1       wonderful little production   filming techni...  positive
2       thought       wonderful way   spend time    ...  positive
3     basically theres   family     little boy jake ...  negative
4     petter matteis love     time   money     visua...  positive
...                                                 ...       ...
1995  feeling minnesota directed   steven baigelmann...  negative
1996    cell 2000 rating 810the cell like antz must ...  positive
1997    movie despite   list   b c     list celebs  ...  negative
1998    loved   movie         could       break     ...  positive
1999        worst movie     ever seen billy zane    ...  negative

[2000 rows x 2 columns]


In [16]:
x = df.iloc[:,0:1]
y = df['sentiment']

In [17]:
x

,review
0,one reviewers mentioned watching ...
1,wonderful little production filming techni...
2,thought wonderful way spend time ...
3,basically theres family little boy jake ...
4,petter matteis love time money visua...
...,...
1995,feeling minnesota directed steven baigelmann...
1996,cell 2000 rating 810the cell like antz must ...
1997,movie despite list b c list celebs ...
1998,loved movie could break ...


In [18]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
y = encode.fit_transform(y)

In [19]:
y

array([1, 1, 1, ..., 0, 1, 0])

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2 , random_state=1)

In [21]:
x_train.shape

(1600, 1)

In [22]:
# applying BOW

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [23]:
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [24]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [25]:
gnb.fit(x_train_bow, y_train)

GaussianNB()

In [26]:
y_pred = gnb.predict(x_test_bow)

from sklearn.metrics import accuracy_score , confusion_matrix
accuracy_score(y_test,y_pred)

0.635

In [27]:
confusion_matrix(y_test,y_pred)

array([[136,  79],
       [ 67, 118]], dtype=int64)

In [28]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(x_train_bow,y_train)
y_pred = rf.predict(x_test_bow)
accuracy_score(y_test, y_pred)

0.78

In [29]:
# using TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer()

In [30]:
x_train_tfidf = tfidf.fit_transform(x_train['review']).toarray()
x_test_tfidf = tfidf.transform(x_test['review']).toarray()

In [31]:
rf = RandomForestClassifier()

rf.fit(x_train_tfidf,y_train)
y_pred = rf.predict(x_test_tfidf)
accuracy_score(y_test, y_pred)

0.8125

In [32]:
# Using Word2Vec


import gensim

from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [34]:
df1 = df

In [35]:
new_df = []

for line in df1['review']:
    raw_sent = sent_tokenize(line)
    for sent in raw_sent:
        new_df.append(simple_preprocess(sent))

In [37]:
print(new_df)

[['one', 'reviewers', 'mentioned', 'watching', 'oz', 'episode', 'youll', 'hooked', 'right', 'exactly', 'happened', 'methe', 'first', 'thing', 'struck', 'oz', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'right', 'word', 'go', 'trust', 'show', 'faint', 'hearted', 'timid', 'show', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'use', 'wordit', 'called', 'oz', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focuses', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'em', 'city', 'home', 'manyaryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'moreso', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'never', 'far', 'awayi', 'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'goes', 'shows', 'wouldnt', 'dare', 'forget', 'pretty', 'pictures', 'painted', 'mainstream

In [38]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 4
)

In [39]:
model.build_vocab(new_df)

In [40]:
model.train(new_df, total_examples= model.corpus_count, epochs= model.epochs)

(1030401, 1157190)

In [41]:
len(model.wv.index_to_key)

14081

In [44]:
def document_vec(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [43]:
document_vec(df['review'].values[0])

array([-2.5998911e-01,  3.7780538e-01,  8.0148853e-02,  3.1038929e-02,
        1.5649466e-01, -7.1249640e-01,  1.8895853e-01,  9.1776592e-01,
       -3.8402972e-01, -2.5788587e-01, -2.7750772e-01, -6.7714548e-01,
       -7.0529535e-02,  1.6558123e-01,  9.6310034e-02, -3.7174580e-01,
        3.0250812e-02, -4.0689003e-01, -2.8089991e-02, -7.2835815e-01,
        2.2368667e-01,  8.1168264e-02,  1.5200382e-01, -2.0569581e-01,
        1.0007310e-01, -2.0559576e-01, -3.1287438e-01, -1.1403549e-01,
       -2.9358649e-01,  1.4882283e-01,  3.5034609e-01,  9.7326323e-02,
        8.8226385e-02, -4.7112542e-01, -1.3555379e-01,  5.4712766e-01,
        1.4074151e-01, -5.1178759e-01, -2.0008004e-01, -8.1378257e-01,
       -6.0900702e-04, -3.9729473e-01,  2.0360680e-02,  1.3058035e-01,
        2.6371178e-01, -1.8795054e-01, -5.8697230e-01,  1.5905557e-02,
        1.7726716e-01,  3.7510619e-01,  2.9036787e-01, -4.7829416e-01,
       -1.2136179e-01, -6.9008239e-02, -3.1036219e-01,  1.9089885e-01,
      

In [45]:
from tqdm import tqdm

In [46]:
x= []
for doc in tqdm(df['review'].values):
    x.append(document_vec(doc))

100%|██████████| 2000/2000 [00:15<00:00, 132.75it/s]


In [47]:
x = np.array(x)

In [48]:
x[0]

array([-2.5998911e-01,  3.7780538e-01,  8.0148853e-02,  3.1038929e-02,
        1.5649466e-01, -7.1249640e-01,  1.8895853e-01,  9.1776592e-01,
       -3.8402972e-01, -2.5788587e-01, -2.7750772e-01, -6.7714548e-01,
       -7.0529535e-02,  1.6558123e-01,  9.6310034e-02, -3.7174580e-01,
        3.0250812e-02, -4.0689003e-01, -2.8089991e-02, -7.2835815e-01,
        2.2368667e-01,  8.1168264e-02,  1.5200382e-01, -2.0569581e-01,
        1.0007310e-01, -2.0559576e-01, -3.1287438e-01, -1.1403549e-01,
       -2.9358649e-01,  1.4882283e-01,  3.5034609e-01,  9.7326323e-02,
        8.8226385e-02, -4.7112542e-01, -1.3555379e-01,  5.4712766e-01,
        1.4074151e-01, -5.1178759e-01, -2.0008004e-01, -8.1378257e-01,
       -6.0900702e-04, -3.9729473e-01,  2.0360680e-02,  1.3058035e-01,
        2.6371178e-01, -1.8795054e-01, -5.8697230e-01,  1.5905557e-02,
        1.7726716e-01,  3.7510619e-01,  2.9036787e-01, -4.7829416e-01,
       -1.2136179e-01, -6.9008239e-02, -3.1036219e-01,  1.9089885e-01,
      

In [49]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

In [50]:
y = encode.fit_transform(df['sentiment'])

In [51]:
y

array([1, 1, 1, ..., 0, 1, 0])

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test , y_train , y_test = train_test_split(x,y, test_size=0.2, random_state=1)

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [54]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)

0.61